# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [125]:
# Load the libraries as required.
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import pickle
import shap



In [126]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [127]:
# Create the features data frame
X = fires_dt.drop(columns='area', axis=1)

In [128]:
# Create the target variable data frame
y = fires_dt['area']   

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [129]:
# Define numerical and categorical columns 
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'] 
categorical_features = ['month', 'day']

# Preprocessor 1: scale numerical columns, one-hot encode categorical columns
preprocessor_1 = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), numeric_features), 
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [130]:
# Apply log1p (log(1 + x)) to reduce skewness
log_transformer = FunctionTransformer(np.log1p, validate=True)

# Build a preprocessing pipeline for numerical variables
num_pipeline = Pipeline(steps=[
    ('log', ColumnTransformer([
        ('log_transform', FunctionTransformer(np.log1p), ['dc', 'dmc'])
    ], remainder='passthrough')),
    ('scaler', StandardScaler())
])

# Preprocessor 2: non-linear transform + scaling + one-hot encoding
preprocessor_2 = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [131]:
# Pipeline A = preproc1 + baseline
pipeline_a = Pipeline(steps=[
    ('preprocessing', preprocessor_1),
    ('regressor', Ridge())
])

In [132]:
# Pipeline B = preproc2 + baseline
pipeline_b = Pipeline(steps=[
    ('preprocessing', preprocessor_2),
    ('regressor', Ridge())
])

In [133]:
# Pipeline C = preproc1 + advanced model
pipeline_c = Pipeline(steps=[
    ('preprocessing', preprocessor_1),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [134]:
# Pipeline D = preproc2 + advanced model
pipeline_d = Pipeline(steps=[
    ('preprocessing', preprocessor_2),
    ('regressor', RandomForestRegressor(random_state=42))
])
    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [135]:
# Define hyperparameter grid for Ridge regression
param_grid_a = {
    'regressor__alpha': [0.01, 0.1, 1.0, 10.0]  # Regularization strength
}

# Run grid search with 5-fold cross-validation
grid_a = GridSearchCV(pipeline_a, param_grid=param_grid_a, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_a.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor__alpha': [0.01, 0.1, 1.0, 10.0]},
             scoring='neg_mean_squared_error')

In [136]:
# Define hyperparameter grid for Ridge regression
param_grid_b = {
    'regressor__alpha': [0.01, 0.1, 1.0, 10.0]  # Regularization strength
}

# Run grid search with 5-fold cross-validation
grid_b = GridSearchCV(pipeline_b, param_grid=param_grid_b, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_b.fit(X, y)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('log',
                                                                                          ColumnTransformer(remainder='passthrough',
                                                                                                            transformers=[('log_transform',
                                                                                                                           FunctionTransformer(func=<ufunc 'log1p'>),
                                                                                                                           ['dc',
                                                                                                                            'dmc'])])),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor__alpha': [0.01, 0.1, 1.0, 10.0]},
             scoring='neg_mean_squared_error')

In [137]:
# Define hyperparameter grid for Random Forest
param_grid_c = {
    'regressor__n_estimators': [50, 100, 150, 200]  # Number of trees in the forest
}

# Run grid search with 5-fold cross-validation
grid_c = GridSearchCV(pipeline_c, param_grid=param_grid_c, cv=5,
                      scoring='neg_mean_squared_error', n_jobs=-1)  # Use all cores

# Fit the grid search to the data
grid_c.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__n_estimators': [50, 100, 150, 200]},
             scoring='neg_mean_squared_error')

In [138]:
# Define hyperparameter grid for max_depth in Random Forest
param_grid_d = {
    'regressor__max_depth': [5, 10, 15, None]  # Maximum depth of the tree
}

# Run grid search with 5-fold cross-validation
grid_d = GridSearchCV(pipeline_d, param_grid=param_grid_d, cv=5,
                      scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_d.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('log',
                                                                                          ColumnTransformer(remainder='passthrough',
                                                                                                            transformers=[('log_transform',
                                                                                                                           FunctionTransformer(func=<ufunc 'log1p'>),
                                                                                                                           ['dc',
                                                                                                                            'dmc'])])),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1, param_grid={'regressor__max_depth': [5, 10, 15, None]},
             scoring='neg_mean_squared_error')

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

In [139]:
# Compare best scores (the higher the better for negative MSE)
results = {
    'Pipeline A': -grid_a.best_score_,
    'Pipeline B': -grid_b.best_score_,
    'Pipeline C': -grid_c.best_score_,
    'Pipeline D': -grid_d.best_score_
}

best_model_name = min(results, key=results.get)
best_model = {'Pipeline A': grid_a, 'Pipeline B': grid_b, 'Pipeline C': grid_c, 'Pipeline D': grid_d}[best_model_name]
print(f"Best model is {best_model_name} with RMSE = {results[best_model_name] ** 0.5:.4f}")

Best model is Pipeline A with RMSE = 65.4114


In [140]:
with open('best_forestfire_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)



# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [141]:
# split into training and test test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# fit the best model on training data
best_pipeline_fitted = best_model.best_estimator_
best_pipeline_fitted.fit(X_train, y_train) # Fit the best pipeline on the training data

# Get transformed feature matrix for SHAP
X_train_transformed = best_pipeline_fitted.named_steps['preprocessing'].transform(X_train)
X_test_transformed = best_pipeline_fitted.named_steps['preprocessing'].transform(X_test)

#Get the trained regressor
final_model = best_pipeline_fitted.named_steps['regressor']

# Store the processed feature names
feature_names = []

#get names for numerical features
feature_names.extend(numeric_features)


try: 
    # Get the OneHotEncoder transformer from the preprocessor
    ohe_transformer = best_pipeline_fitted.named_steps['preprocessing'].named_transformers_['cat'] 
    # Get the new feature names generated by OneHotEncoder
    ohe_feature_names = ohe_transformer.get_feature_names_out(categorical_features)
    # Add OHE feature names to the list
    feature_names.extend(ohe_feature_names.tolist())

except KeyError: 
    # Handle case where 'cat' key for OneHotEncoder is not found
    print("OneHotEncoder transformer not found under 'cat' name in preprocessor. C")
    feature_names = None

except AttributeError:
    # Handle case if get_feature_names_out method is unavailable
    print("OneHotEncoder does not have get_feature_names_out. ")
    feature_names = None 

if feature_names is not None:
    # Print the first 10 constructed feature names for verification
    print(f"Feature names used for SHAP: {feature_names[:10]}...")
else:
    # Inform user if automatic construction failed
    print("Could not construct feature names automatically.")

if feature_names is not None:
    # Print the first 10 constructed feature names for verification
    print(f"Feature names used for SHAP: {feature_names[:10]}...")
else:
    # Inform user if automatic construction failed
    print("Could not construct feature names automatically.")



Feature names used for SHAP: ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']...
Feature names used for SHAP: ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']...


For a specific fire incident (e.g., index 50 in the test set), the model predicted 12.5 hectares vs. an average of 8.3 hectares, with temp and ISI increasing the prediction and rain and RH lowering it. Across the training set, temp, ISI, and FFMC were most important, while rain and day were least important. I’d consider removing rain, day, and RH to simplify the model, then re-train, re-evaluate RMSE on the test set, and use cross-validation to confirm performance impact.

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.